In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [5]:
# read the dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [6]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
['Age','Experience'] ---> new feature --->independent feature

In [10]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Feature")

In [11]:
output = featureAssembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [14]:
finalized_data = output.select("Independent Feature","Salary")

In [15]:
finalized_data.show()
# salary is dependent feature.

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Feature',labelCol='Salary')
regressor = regressor.fit(train_data)

In [21]:
# Coefficient
regressor.coefficients

DenseVector([-90.9091, 1272.7273])

In [22]:
# Intercept
regressor.intercept

17545.454545455385

In [23]:
# Prediction
pred_result = regressor.evaluate(test_data)

In [24]:
pred_result.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|16909.090909091006|
|         [24.0,3.0]| 20000|19181.818181818242|
|        [31.0,10.0]| 30000|27454.545454545525|
+-------------------+------+------------------+



In [25]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(1757.5757575757464, 3597796.1432506596)